In [1]:
%pip install -U langchain langchain-core langchain-google-genai langchain-community langgraph python-dotenv 


  Attempting uninstall: langchain-core

    Found existing installation: langchain-core 1.0.4

    Uninstalling langchain-core-1.0.4:

      Successfully uninstalled langchain-core-1.0.4

   ---------------------------------------- 0/3 [langchain-core]
   ---------------------------------------- 0/3 [langchain-core]
   ---------------------------------------- 0/3 [langchain-core]
   ---------------------------------------- 0/3 [langchain-core]
   ---------------------------------------- 0/3 [langchain-core]
   ---------------------------------------- 0/3 [langchain-core]
   ---------------------------------------- 0/3 [langchain-core]
   ---------------------------------------- 0/3 [langchain-core]
   ---------------------------------------- 0/3 [langchain-core]
   ---------------------------------------- 0/3 [langchain-core]
   ---------------------------------------- 0/3 [langchain-core]
   ---------------------------------------- 0/3 [langchain-core]
   ----------------------------

In [2]:
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv
import os

# --- 환경 설정 ---
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

if not GOOGLE_API_KEY:
    raise ValueError("환경 변수 'GOOGLE_API_KEY'를 설정해주세요. (.env 파일 확인)")
model = GoogleGenerativeAI(model="gemini-2.5-flash")
model.invoke("안녕하세요!")

'안녕하세요! 만나서 반갑습니다. 😊\n\n무엇을 도와드릴까요?'

In [6]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    """
    State 클래스는 TypedDict을 상속받습니다.

    속성:
        messages (Annotated[list[str], add_messages]): 메시지들은 "list" 타입을 가집니다.
        'add_messages' 함수는 이 상태 키가 어떻게 업데이트되어야 하는지를 정의합니다.
        (이 경우, 메시지를 덮어쓰는 대신 리스트에 추가합니다)
    """
    messages: Annotated[list[str], add_messages]

graph_builder = StateGraph(State)

In [8]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

# --- [수정] .env 및 Tavily API 키 로드를 위한 import ---
from dotenv import load_dotenv
import os
from langchain_community.tools.tavily_search import TavilySearchResults

import bs4
from langchain_community.document_loaders import WebBaseLoader

# --- [추가] .env 파일에서 API 키 로드 ---
load_dotenv()
tavily_api_key = os.getenv("TAVILY_API_KEY")

if not tavily_api_key:
    print("경고: .env 파일에 TAVILY_API_KEY가 설정되지 않았습니다.")
# ------------------------------------


# 도구 함수 정의
@tool
def get_current_time(timezone: str, location: str) -> str:
    """현재 시각을 반환하는 함수."""
    try:
        tz = pytz.timezone(timezone)
        now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
        result = f'{timezone} ({location}) 현재시각 {now}'
        # print(result)
        return result
    except pytz.UnknownTimeZoneError:
        return f"알 수 없는 타임존: {timezone}"
    

# --- [수정] get_web_search 함수를 Tavily 버전으로 교체 ---
@tool
def get_web_search(query: str) -> str:
    """
    Tavily를 사용하여 웹 검색을 수행하는 함수.
    'search_period' 인자는 받지 않습니다.
    
    Args:
        query (str): 검색어

    Returns:
        str: 검색 결과 (Snippet, Title, URL 형식)
    """
    print('\n-------- TAVILY WEB SEARCH --------')
    print(f'Query: {query}')

    # Tavily 검색 도구 초기화 (k=5: 5개의 결과 요청)
    search = TavilySearchResults(
        k=5, 
        tavily_api_key=tavily_api_key # API 키 명시적 전달
    ) 
    
    try:
        # Tavily는 [ {'url': ..., 'content': ...}, ... ] 형식의 딕셔너리 리스트를 반환
        docs = search.invoke(query)
        
        results_str_list = []
        
        # (원본 코드의 print 루프 동작을 유지)
        print("Search Results:")
        for i, doc in enumerate(docs):
            snippet = doc.get('content', '내용 없음')
            title = doc.get('title', '제목 없음')
            url = doc.get('url', '출처 없음')
            
            # (참고 코드의 반환 형식)
            formatted_result = f"Snippet: {snippet}\nTitle: {title}\nURL: {url}"
            results_str_list.append(formatted_result)
            
            # (원본 코드의 print 동작)
            print(f"--- Result {i+1} ---")
            print(formatted_result)

        # (참고 코드의 반환 방식)
        return "\n\n;\n\n".join(results_str_list)

    except Exception as e:
        print(f"Tavily 검색 오류: {e}")
        return f"검색 중 오류가 발생했습니다: {e}"


# 도구 바인딩 (변경 없음)
tools = [get_current_time, get_web_search]

In [9]:
tools[0].invoke({"timezone": "Asia/Seoul", "location":"서울"})

'Asia/Seoul (서울) 현재시각 2025-11-17 20:22:17'

In [10]:
tools[1].invoke({"query": "전지적 독자 시점 영화"})


-------- TAVILY WEB SEARCH --------
Query: 전지적 독자 시점 영화


C:\Users\ziclp\AppData\Local\Temp\ipykernel_15608\338524954.py:53: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search = TavilySearchResults(


Search Results:
--- Result 1 ---
Snippet: 2) 어떤 이야기인가요? [전지적 독자 시점]은 포스트 아포칼립스 배경의 현대 판타지 영화다. 예고편에 나와 있는 내용 수준에서 배경을 설명한다. 더럽
Title: 영화 전지적 독자 시점 리뷰 - 영화 추천 여부, 각색 등 이 영화에 대한 ...
URL: https://blog.naver.com/aramiroo/223952006946
--- Result 2 ---
Snippet: 결론적으로 '전지적 독자 시점'은 거대한 원작의 명성과 막대한 자본에 기댔을 뿐, 정작 한 편의 영화로서 갖춰야 할 기본적인 완성도를 놓친 작품입니다. 조악한 시각 효과와 힘없는 액션, 그리고 방향성을 상실한 서사는 그 어떤 관객층도 만족시키기 어려운 결과물을 낳았습니다.

영화는 대놓고 후속작을 암시하며 막을 내리지만, 과연 이러한 완성도로 675만 명이라는 손익분기점을 넘어 다음 이야기를 이어갈 수 있을지 의문이 듭니다. 한국 영화계가 어렵다는 이야기가 연일 흘러나오는 지금, '전지적 독자 시점'의 도전은 무모함을 넘어 무책임하게까지 느껴집니다. 300억 원이 넘는 막대한 예산을 허무하게 스크린 위에서 터트려버린 이 영화는, 한국형 블록버스터의 실패 사례로 오래도록 기억될 것 같습니다. 부디 제2의 '성냥팔이 소녀의 재림'이 되지 않기를 바랄 뿐입니다. [...] 애기가 되어버린 나의 엄마“엄마! 이거 어떻게 해야 해?”내가 아주 어린 애기였을 때부터, 나에게 엄마는 모르는 게 하나도 없는 세상에서 제일 똑똑한 사람이었다. 무엇을 물어봐도 엄마는 다 대답을 해주었고 나에게 어떠한 문제가 생겨도 엄마의 도움으로 해결할 수 있었다. 그래서 나는 엄마를 자주 찾았다. 숙제를 할 때, 티비가 안 나올 때, 입고 싶은 옷을 못 찾을 때……. 언 by 송이
   디자이너 아내와 개발자 남편이 같이 일할 때사내 연애로 결혼한 지 2달, 만난 지는 1년 5개월. 짧은 시간인지라 우리는 싸운 적도 별로 없다. 다섯 손가락 

'Snippet: 2) 어떤 이야기인가요? [전지적 독자 시점]은 포스트 아포칼립스 배경의 현대 판타지 영화다. 예고편에 나와 있는 내용 수준에서 배경을 설명한다. 더럽\nTitle: 영화 전지적 독자 시점 리뷰 - 영화 추천 여부, 각색 등 이 영화에 대한 ...\nURL: https://blog.naver.com/aramiroo/223952006946\n\n;\n\nSnippet: 결론적으로 \'전지적 독자 시점\'은 거대한 원작의 명성과 막대한 자본에 기댔을 뿐, 정작 한 편의 영화로서 갖춰야 할 기본적인 완성도를 놓친 작품입니다. 조악한 시각 효과와 힘없는 액션, 그리고 방향성을 상실한 서사는 그 어떤 관객층도 만족시키기 어려운 결과물을 낳았습니다.\n\n영화는 대놓고 후속작을 암시하며 막을 내리지만, 과연 이러한 완성도로 675만 명이라는 손익분기점을 넘어 다음 이야기를 이어갈 수 있을지 의문이 듭니다. 한국 영화계가 어렵다는 이야기가 연일 흘러나오는 지금, \'전지적 독자 시점\'의 도전은 무모함을 넘어 무책임하게까지 느껴집니다. 300억 원이 넘는 막대한 예산을 허무하게 스크린 위에서 터트려버린 이 영화는, 한국형 블록버스터의 실패 사례로 오래도록 기억될 것 같습니다. 부디 제2의 \'성냥팔이 소녀의 재림\'이 되지 않기를 바랄 뿐입니다. [...] 애기가 되어버린 나의 엄마“엄마! 이거 어떻게 해야 해?”내가 아주 어린 애기였을 때부터, 나에게 엄마는 모르는 게 하나도 없는 세상에서 제일 똑똑한 사람이었다. 무엇을 물어봐도 엄마는 다 대답을 해주었고 나에게 어떠한 문제가 생겨도 엄마의 도움으로 해결할 수 있었다. 그래서 나는 엄마를 자주 찾았다. 숙제를 할 때, 티비가 안 나올 때, 입고 싶은 옷을 못 찾을 때……. 언 by 송이\n   디자이너 아내와 개발자 남편이 같이 일할 때사내 연애로 결혼한 지 2달, 만난 지는 1년 5개월. 짧은 시간인지라 우리는 싸운 적도 별로 없다. 다섯 손가락 안에 꼽힌다고 하면 적은 걸까. 꼽힌 일들도 양방의 싸

In [11]:
tools[1].invoke({"query": "전지적 독자 시점 영화"})


-------- TAVILY WEB SEARCH --------
Query: 전지적 독자 시점 영화
Search Results:
--- Result 1 ---
Snippet: 2) 어떤 이야기인가요? [전지적 독자 시점]은 포스트 아포칼립스 배경의 현대 판타지 영화다. 예고편에 나와 있는 내용 수준에서 배경을 설명한다. 더럽
Title: 영화 전지적 독자 시점 리뷰 - 영화 추천 여부, 각색 등 이 영화에 대한 ...
URL: https://blog.naver.com/aramiroo/223952006946
--- Result 2 ---
Snippet: 결론적으로 '전지적 독자 시점'은 거대한 원작의 명성과 막대한 자본에 기댔을 뿐, 정작 한 편의 영화로서 갖춰야 할 기본적인 완성도를 놓친 작품입니다. 조악한 시각 효과와 힘없는 액션, 그리고 방향성을 상실한 서사는 그 어떤 관객층도 만족시키기 어려운 결과물을 낳았습니다.

영화는 대놓고 후속작을 암시하며 막을 내리지만, 과연 이러한 완성도로 675만 명이라는 손익분기점을 넘어 다음 이야기를 이어갈 수 있을지 의문이 듭니다. 한국 영화계가 어렵다는 이야기가 연일 흘러나오는 지금, '전지적 독자 시점'의 도전은 무모함을 넘어 무책임하게까지 느껴집니다. 300억 원이 넘는 막대한 예산을 허무하게 스크린 위에서 터트려버린 이 영화는, 한국형 블록버스터의 실패 사례로 오래도록 기억될 것 같습니다. 부디 제2의 '성냥팔이 소녀의 재림'이 되지 않기를 바랄 뿐입니다. [...] 애기가 되어버린 나의 엄마“엄마! 이거 어떻게 해야 해?”내가 아주 어린 애기였을 때부터, 나에게 엄마는 모르는 게 하나도 없는 세상에서 제일 똑똑한 사람이었다. 무엇을 물어봐도 엄마는 다 대답을 해주었고 나에게 어떠한 문제가 생겨도 엄마의 도움으로 해결할 수 있었다. 그래서 나는 엄마를 자주 찾았다. 숙제를 할 때, 티비가 안 나올 때, 입고 싶은 옷을 못 찾을 때……. 언 by 송이
   디자이너 아내와 개발자 남편이 같이 일할 때사내 연애로 

'Snippet: 2) 어떤 이야기인가요? [전지적 독자 시점]은 포스트 아포칼립스 배경의 현대 판타지 영화다. 예고편에 나와 있는 내용 수준에서 배경을 설명한다. 더럽\nTitle: 영화 전지적 독자 시점 리뷰 - 영화 추천 여부, 각색 등 이 영화에 대한 ...\nURL: https://blog.naver.com/aramiroo/223952006946\n\n;\n\nSnippet: 결론적으로 \'전지적 독자 시점\'은 거대한 원작의 명성과 막대한 자본에 기댔을 뿐, 정작 한 편의 영화로서 갖춰야 할 기본적인 완성도를 놓친 작품입니다. 조악한 시각 효과와 힘없는 액션, 그리고 방향성을 상실한 서사는 그 어떤 관객층도 만족시키기 어려운 결과물을 낳았습니다.\n\n영화는 대놓고 후속작을 암시하며 막을 내리지만, 과연 이러한 완성도로 675만 명이라는 손익분기점을 넘어 다음 이야기를 이어갈 수 있을지 의문이 듭니다. 한국 영화계가 어렵다는 이야기가 연일 흘러나오는 지금, \'전지적 독자 시점\'의 도전은 무모함을 넘어 무책임하게까지 느껴집니다. 300억 원이 넘는 막대한 예산을 허무하게 스크린 위에서 터트려버린 이 영화는, 한국형 블록버스터의 실패 사례로 오래도록 기억될 것 같습니다. 부디 제2의 \'성냥팔이 소녀의 재림\'이 되지 않기를 바랄 뿐입니다. [...] 애기가 되어버린 나의 엄마“엄마! 이거 어떻게 해야 해?”내가 아주 어린 애기였을 때부터, 나에게 엄마는 모르는 게 하나도 없는 세상에서 제일 똑똑한 사람이었다. 무엇을 물어봐도 엄마는 다 대답을 해주었고 나에게 어떠한 문제가 생겨도 엄마의 도움으로 해결할 수 있었다. 그래서 나는 엄마를 자주 찾았다. 숙제를 할 때, 티비가 안 나올 때, 입고 싶은 옷을 못 찾을 때……. 언 by 송이\n   디자이너 아내와 개발자 남편이 같이 일할 때사내 연애로 결혼한 지 2달, 만난 지는 1년 5개월. 짧은 시간인지라 우리는 싸운 적도 별로 없다. 다섯 손가락 안에 꼽힌다고 하면 적은 걸까. 꼽힌 일들도 양방의 싸

In [12]:
for tool in tools:
    print(tool.name, tool)

get_current_time name='get_current_time' description='현재 시각을 반환하는 함수.' args_schema=<class 'langchain_core.utils.pydantic.get_current_time'> func=<function get_current_time at 0x0000022744CFF560>
get_web_search name='get_web_search' description="Tavily를 사용하여 웹 검색을 수행하는 함수.\n'search_period' 인자는 받지 않습니다.\n\nArgs:\n    query (str): 검색어\n\nReturns:\n    str: 검색 결과 (Snippet, Title, URL 형식)" args_schema=<class 'langchain_core.utils.pydantic.get_web_search'> func=<function get_web_search at 0x0000022744CFFCE0>


In [13]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
import os
from langchain_community.tools.tavily_search import TavilySearchResults

# --- 환경 설정 ---
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

if not GOOGLE_API_KEY:
    raise ValueError("환경 변수 'GOOGLE_API_KEY'를 설정해주세요. (.env 파일 확인)")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=GOOGLE_API_KEY,
)

In [14]:
for tool in tools:
    print(tool.name, tool)

get_current_time name='get_current_time' description='현재 시각을 반환하는 함수.' args_schema=<class 'langchain_core.utils.pydantic.get_current_time'> func=<function get_current_time at 0x0000022744CFF560>
get_web_search name='get_web_search' description="Tavily를 사용하여 웹 검색을 수행하는 함수.\n'search_period' 인자는 받지 않습니다.\n\nArgs:\n    query (str): 검색어\n\nReturns:\n    str: 검색 결과 (Snippet, Title, URL 형식)" args_schema=<class 'langchain_core.utils.pydantic.get_web_search'> func=<function get_web_search at 0x0000022744CFFCE0>


In [15]:
model_with_tools=llm.bind_tools(tools) #gemini는 chatGoogleGenerativeAI를 llm으로 받음

def generate(state: State):
    """
    주어진 상태를 기반으로 챗봇의 응답 메시지를 생성합니다.
    
    매개변수:
    state (State): 현재 대화 상태를 나태는 객체로, 이전 메시지들이 포함되어 있습니다.

    반환값:
    dict: 모델이 생성한 응답 메시지를 포함하는 딕셔너리
          형식은 {"messages": [응답 메시지]}입니다.
    """

    return {"messages": model_with_tools.invoke(state["messages"])}

graph_builder.add_node("generate", generate)

In [ ]:
import json
from langchain_core.messages import ToolMessage

class BasicToolNode:
    """
    도구를 실행하는 노드 클래스입니다. 마지막 AIMessage에서 요청된 도구를 실행합니다.
    ...
    """
    """A node that runs the tools requested in the last AIMessage."""

    # ✅ 클래스 내부에 들여쓰기 (4칸 공백 권장)
    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    # ✅ 클래스 내부에 들여쓰기 (4칸 공백 권장)
    def __call__(self, inputs: dict):
        if messages := inputs.get("messages",[]):
            # inputs에 messages가 있으면 messages를 가져오고 없으면 빈 리스트 가져오기
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        outputs = []

        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"]
                )
            )
        return {"messages": messages + outputs}

# 이제 __init__이 인자(tools)를 받는 클래스 메서드임을 인식하므로 오류가 해결됩니다.
tool_node = BasicToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

TypeError: BasicToolNode() takes no arguments